## Предсказание стоимости жилья

**Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.**

### 1. Загрузим и изучим данные

In [66]:
# Импортируем библиотеки

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [31]:
# Читаем файл csv

df = pd.read_csv('/Users/danil/Desktop/Data Science/Яндекс Практикум Data Science/10/[SW.BAND] [Яндекс.Практикум] Специалист по Data Science Плюс. Часть 10 из 16 (2022)/[SW.BAND] 06 Самостоятельный проект/insurance.csv')

In [32]:
# Изучим данные

print('Данные в таблице')
display(df.head())
print('\n', 'Информация о таблице', '\n')
display(df.info())
print('\n', 'Колличество пропусков', '\n')
display(df.isna().sum())
print('\n', 'Колличество дубликатов:', df.duplicated().sum(), '\n')
print('\n', 'Колличество уникальных значений', '\n')
display(df.nunique())

Данные в таблице


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



 Информация о таблице 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None


 Колличество пропусков 



Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


 Колличество дубликатов: 153 


 Колличество уникальных значений 



Пол                    2
Возраст               46
Зарплата             524
Члены семьи            7
Страховые выплаты      6
dtype: int64

In [33]:
# Удалим дубликаты

df.drop_duplicates(inplace=True)

### 2. Ответим на вопрос и обоснуем решение

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
1. Изменится. Приведите примеры матриц.
2. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула расчета весов для линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

Представим новую матрицу признаков $X1$ как произведение старой $X$ на матрицу $P$ (обратимую):

$$
X1 = X * P
$$

Подставим новое значение X1 в формулу $w1$:

$$
w1 = ((X P)^T X P)^{-1} (X P)^T y
$$

Раскроем первое произведение $ (XP)^T $:

$$
w1 = (P^T X^T X P)^{-1} P^T X^T y
$$

Перегруппируем множители в скобках, чтобы это выглядело как произведение трех множителей:

$$
w1 = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно $E$ (единичная матрица):

$$
w1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Можно заметить, что справа получилась формула для $w$:

$$
w1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

Подставим новое значение весов  𝑤1  в формулу для предсказаний линейной регрессии.
Предсказания модели расчитываются по формуле:

$$
a = Xw
$$

Подставим в эту формулу значения $X1$ и $w1$ для того, чтобы на их основе вычислить предсказания модели $a1$:

$$
a1 = X1w1 = XPP^{-1} w
$$

Так как $P$ по условию обратимая, то произведение $PP^{-1}$ снова равно $E$:

$$
a1 = XPP^{-1} w = X E w = X w = a
$$

Мы доказали, что предсказания $a1$ для матрицы признаков, умноженных на обратимую матрицу $P$ равны предсказаниям $a$.<br>
Следовательно, отвечая на вопрос изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу - нет, не изменится. В данном случае, параметры линейной регрессии в исходной задаче $w$ и в преобразованной $w1$ связаны следующим образом:

$$
w1 = P^{-1} w
$$

### Вывод:
При умножении признаков на обратимую матрицу, предсказания модели не поменяется, но поменяются веса для новых признаков. Придется переобучать модель на новых признаках.

### 3. Предложим алгоритм преобразования данных для решения задачи. Обоснуем, почему качество линейной регрессии не поменяется.

In [36]:
# Создадим список столбцов для масштабирования
# Выделим признаки и целевой признак

numeric_list = [
    'Возраст', 'Зарплата', 
    'Члены семьи'
]

scaler = StandardScaler()
scaler.fit(df[numeric_list])
df[numeric_list] = scaler.transform(df[numeric_list])

features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [59]:
# Создадим рандомную матрицу размером в колличество признаков
# Проверим ее на обратность

P = np.random.normal(loc=0.0, scale=1.0, size=(4, 4))
P_reverse = np.linalg.inv(P)
P_reverse

array([[-0.58405044,  0.51697223,  0.52253328, -0.4106839 ],
       [ 0.56403724, -1.47517808, -0.55669972,  0.76944328],
       [ 0.31942415, -0.68210714, -0.28177901,  0.89674179],
       [ 1.42273442, -1.15990133,  0.20433398,  1.98814826]])

In [37]:
# Создадим класс линейной ргерессии 
# и проверим на нем показатель R2

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
            
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4302010044852068


In [80]:
# Проверим на том же классе признаки умноженные на рандомную матрицу

model.fit(features.dot(P), target)
predictions = model.predict(features.dot(P))
print(r2_score(target, predictions))

0.4302010044852067


### Вывод

Алгоритмом действия стало создание рандомной матрицы размером 4х4 и умножение ее на признаки перед обучением. Сравнение показателя R2 показало, что качество модели почти не ухудшилось в сравнении с обучением на не измененных признаках.

### 4. Запрограммируем этот алгоритм, применив матричные операции. Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применим метрику R2.

In [81]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions))

MSE Score: 0.12527263826681606
R2_Score для преобразованных  признаков:  0.4302010044852066


In [82]:
model.fit(features.dot(P), target)
predictions = model.predict(features.dot(P))
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions))

MSE Score: 0.12527263826681603
R2_Score для преобразованных  признаков:  0.4302010044852067


<div class="alert alert-block alert-info">
    <ul type='circle'>
        
### Общий вывод
        
**Мы доказали как математически, так и на модели, что данные пользователей можно защитить перемножив их на обратимую матрицу. При этом качество модели не изменится.**